In [ ]:
import pandas as pd
import spacy

In [ ]:
# sample spacy similarity
nlp = spacy.load('en')  # make sure to use larger model -> later
tokens = nlp(u'dog cat banana')

for token1 in tokens:
    for token2 in tokens:
        print(token1.text, token2.text, token1.similarity(token2))

In [ ]:
# read all CSV files
pred_ents = pd.read_csv("predictions.csv")
# reviews = pd.read_csv("all_reviews.csv")
# menus = pd.read_csv("all_reviews.csv")

In [ ]:
# combine reviews and annotations; drop unnecessary cols
print("predicted entities = ",pred_ents.shape)
print("reviews = ",reviews.shape)
reviews_with_ents = reviews.merge(right = pred_ents,how = 'inner',on ='review_id', sort = True)
print("Merged = ", reviews_with_ents.shape)
reviews_with_ents.head()

In [ ]:
reviews_with_ents.drop(['user_name', 'business_id_y'], axis = 1, inplace = True)
reviews_with_ents.columns = ['review_id', 'business_id', 'stars', 'date', 'text', 'entities']
reviews_with_ents.head()

In [ ]:
pred_ents['entities'][0]

In [ ]:
reviews['sentences'] = reviews['text'].apply(lambda x: x.split('.'))
reviews['sentences'] = reviews['sentences'].apply(lambda x: [s.strip() for s in x])


In [ ]:
s = reviews.apply(lambda x: pd.Series(x['sentences']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'sentence'
sent_tokenize_reviews = reviews.drop('sentences', axis=1).join(s).drop('text', axis = 1)
sent_tokenize_reviews.shape
sent_pred_ents = pd.read_csv("predictions_sentence.csv")

In [ ]:
from ast import literal_eval
sent_pred_ents = pd.read_csv("predictions_sentence.csv")
sent_pred_ents['entities'] = sent_pred_ents['entities'].apply(literal_eval)
sent_pred_ents['len'] = sent_pred_ents['entities'].apply(lambda x: len(x))
agg1 = sent_pred_ents.groupby(['review_id', 'business_id']).agg({'len' : 'sum'})

In [ ]:
sent_pred_ents

In [ ]:
pred_ents['entities'] = pred_ents['entities'].apply(literal_eval)
pred_ents['len'] = pred_ents['entities'].apply(lambda x: len(x))
agg2 = pred_ents.groupby(['review_id','business_id']).agg({'len':'sum'})
agg2

In [ ]:
m = agg1.merge(right = agg2, how = 'inner', on = 'review_id')


In [ ]:
m['len_x'] = pd.to_numeric(m['len_x'])
m['len_y'] = pd.to_numeric(m['len_y'])
m['check'] = abs(m['len_x'] - m['len_y'])
print(sum(m['len_x']))
print(sum(m['len_y']))

In [ ]:
data = [['Alex',10],['Bob',12],['Clarke',13]]
df = pd.DataFrame(data,columns=['Name','Age'])
df

In [ ]:
large_diff = m[m['check'] > 3]
s = large_diff['review_id'].tolist()

In [ ]:
sent_pred_ents[sent_pred_ents['review_id'].isin(s)].to_csv("new_missing.csv")

## Similarity Between Items

In [ ]:
from ast  import literal_eval
pred_ann_snt = pd.read_csv("predictions_sentence_sentiment.csv", encoding = 'utf8')
pred_ann_snt['entities'] = pred_ann_snt['entities'].apply(literal_eval)
pred_ann_snt['len'] = pred_ann_snt['entities'].apply(lambda x: len(x))

print(pred_ann_snt.shape)
pred_ann_snt.drop(pred_ann_snt[pred_ann_snt.len < 1].index, inplace=True)
pred_ann_snt.drop(columns='len', inplace=True)
print(pred_ann_snt.shape)

## Creating a menu

In [ ]:
menu_dict = {}
unique_b_id = pred_ann_snt['business_id'].unique()
unique_b_id

pred_ann_snt['food'] = pred_ann_snt['entities'].apply(lambda x: [a[0] for a in x])
pred_ann_snt.head()

for b_id in unique_b_id:
    menu_dict[b_id] = pred_ann_snt[pred_ann_snt['business_id'] == b_id]['food'].values.tolist()

for k in menu_dict.keys():
    print(k)
    items = set()
    for l in menu_dict.get(k, ""):
#         print(l)
        items = items.union(set(l))
    menu_dict[k] = list(items)

    
# similarity with  spacy
sim_list = []
for token1 in tokens:
    for token2 in tokens:
        sim_list.append((token1.text, token2.text, token1.similarity(token2)))

# from itertools import ifilter
filtered = [item for item in sim_list if item[2]>0.8]

In [ ]:
import csv
with open("filtered_sim_list.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(filtered)

## Similarity with substring match

In [ ]:
for k in menu_dict.keys():
    menu = menu_dict.get(k, "")
    print(len(menu))
    for item1 in menu:
        for item2 in menu:
            if len(item1) > len(item2) and item2 in item1:
                if item2 in menu:
                    menu.remove(item2)
            elif len(item2) > len(item1) and item1 in item2:
                if item1 in menu:
                    menu.remove(item1)
    menu_dict[k] = menu

#substring match
import pickle
pickle.dump(menu_dict, open( "menu_dict_substring_match.p", "wb" ))

## fuzzy match

In [ ]:
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

#create a raw menu from mentions
menu_dict_fuzz = {}
unique_b_id = pred_ann_snt['business_id'].unique()
unique_b_id

pred_ann_snt['food'] = pred_ann_snt['entities'].apply(lambda x: [a[0] for a in x])
pred_ann_snt.head()

for b_id in unique_b_id:
    menu_dict_fuzz[b_id] = pred_ann_snt[pred_ann_snt['business_id'] == b_id]['food'].values.tolist()

for k in menu_dict_fuzz.keys():
#     print(k)
    items = set()
    for l in menu_dict_fuzz.get(k, ""):
        items = items.union(set(l))
    menu_dict_fuzz[k] = list(items)

print("Before Match!")
for k in menu_dict_fuzz.keys():
    print(k, len(menu_dict_fuzz.get(k, "")))
    
for k in menu_dict_fuzz.keys():
    menu = menu_dict_fuzz.get(k, "")
#     print(len(menu))
    for item1 in menu:
        for item2 in menu:
            if fuzz.ratio(item1,item2)  > 80 and item1 != item2:
#                 print(item1," , ", item2, " score = ", fuzz.ratio(item1, item2))
                if len(item1) > len(item2):
                    if item2 in menu:
                        menu.remove(item2)
                elif len(item2) > len(item1):
                    if item1 in menu:
                        menu.remove(item1)
    menu_dict_fuzz[k] = menu

print("After match!")
for k in menu_dict_fuzz.keys():
    print(k, len(menu_dict_fuzz.get(k, "")))

In [17]:
#substring match
import pickle
file = open( "menu_dict_fuzzy_match.p", "wb" )
pickle.dump(menu_dict_fuzz, file)
file.close()

NameError: name 'menu_dict_fuzz' is not defined

## No similarity match!

In [ ]:
#create a raw menu from mentions
menu_dict_plain = {}
unique_b_id = pred_ann_snt['business_id'].unique()
unique_b_id

pred_ann_snt['food'] = pred_ann_snt['entities'].apply(lambda x: [a[0] for a in x])
pred_ann_snt.head()

for b_id in unique_b_id:
    menu_dict_plain[b_id] = pred_ann_snt[pred_ann_snt['business_id'] == b_id]['food'].values.tolist()

for k in menu_dict_fuzz.keys():
#     print(k)
    items = set()
    for l in menu_dict_plain.get(k, ""):
        items = items.union(set(l))
    menu_dict_plain[k] = list(items)

## Scoring metric

In [ ]:
for k in actual_menus.keys():
    print("Business id: ", k)
    menu_size = len(actual_menus.get(k, ""))
    fuzz_count = 0
    substr_count = 0
    plain_count = 0 
    fuzz_count += len(set(actual_menus.get(k, "")) & set(menu_dict_fuzz.get(k, "")))
    substr_count += len(set(actual_menus.get(k, "")) & set(menu_dict.get(k, "")))
    plain_count += len(set(actual_menus.get(k, "")) & set(menu_dict_plain.get(k, "")))
    print("Menu item found with fuzzy match - ", fuzz_count/menu_size*100)
    print("Menu item found with substring match - ", substr_count/menu_size*100)
    print("Menu item found with no similiarity match - ", plain_count/menu_size*100)

### Top K items

In [1]:
import pandas as pd
from ast  import literal_eval
pred_ann_snt = pd.read_csv("predictions_sentence_sentiment.csv", encoding = 'utf8')
pred_ann_snt['entities'] = pred_ann_snt['entities'].apply(literal_eval)
pred_ann_snt['len'] = pred_ann_snt['entities'].apply(lambda x: len(x))

print(pred_ann_snt.shape)
pred_ann_snt.drop(pred_ann_snt[pred_ann_snt.len < 1].index, inplace=True)
pred_ann_snt.drop('len',axis= 1, inplace=True)
print(pred_ann_snt.shape)


(35869, 10)
(11298, 9)


In [2]:
reviews = pd.read_csv("all_reviews.csv")
pred_ann_snt = pred_ann_snt.merge(right = reviews, how ="left", on = ["review_id", "business_id"])
pred_ann_snt.drop(['compound', 'neg', 'pos', 'neu', 'user_name', 'text','sentence'], axis = 1, inplace=True)
pred_ann_snt.head()

,review_id,business_id,entities,vader_sentiment,stars,date
0,1,1118,"[(curry, 102, 107)]",negative,3,10/30/18
1,1,1118,"[(channa bhatura curry, 6, 26), (curry, 59, 64)]",positive,3,10/30/18
2,1,1118,"[(soup, 4, 8)]",positive,3,10/30/18
3,1,1118,"[(seemingly, 125, 134)]",positive,3,10/30/18
4,2,1118,"[(cuisine, 31, 38)]",positive,4,10/11/18


In [3]:
pred_ann_snt['food'] = pred_ann_snt['entities'].apply(lambda x: [a[0] for a in x])
pred_ann_snt.drop('entities', axis = 1, inplace = True)

In [4]:
pred_ann_snt['date'] = pd.to_datetime(pred_ann_snt['date'])
pred_ann_snt['today'] = "12/6/2018" #today's date
pred_ann_snt['today'] = pd.to_datetime(pred_ann_snt['today'])
pred_ann_snt['sentiment_score'] = pred_ann_snt['vader_sentiment'].apply(lambda x: 1 if x == "positive" else 0)
pred_ann_snt['rating'] = pred_ann_snt['stars'].apply(lambda x: 1 if x>3 else 0)
pred_ann_snt['recency'] = pred_ann_snt['today'] - pred_ann_snt['date']

In [5]:
pred_ann_snt.drop(['vader_sentiment', 'stars', 'date','today'], axis = 1, inplace = True)
pred_ann_snt.head()

,review_id,business_id,food,sentiment_score,rating,recency
0,1,1118,[curry],0,0,37 days
1,1,1118,"[channa bhatura curry, curry]",1,0,37 days
2,1,1118,[soup],1,0,37 days
3,1,1118,[seemingly],1,0,37 days
4,2,1118,[cuisine],1,1,56 days


In [6]:
s = pred_ann_snt.apply(lambda x: pd.Series(x['food']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'dish'
scored_df = pred_ann_snt.drop('food', axis=1).join(s)

In [7]:
import math
lambda_val = 0
scored_df['score'] = (scored_df['sentiment_score']+scored_df['rating'])/2
# scored_df['recency'] = pd.to_numeric(scored_df['recency'])
scored_df['score_1'] = scored_df['recency'].apply(lambda x: math.exp(-lambda_val*x.days))
scored_df['score'] = scored_df['score']*scored_df['score_1']

In [8]:
scored_df['dish'][196]

196         pepper shrimp
196    incredible chicken
196          pork curries
Name: dish, dtype: object

In [9]:
new_df = scored_df.groupby(['business_id', 'dish']).agg({'score':'sum'}).reset_index()

In [11]:
new_df.head()

,business_id,dish,score
0,1111,& lobster newburg,0.5
1,1111,abraham,2.0
2,1111,aged ribeye,1.0
3,1111,aioli,0.5
4,1111,alas,0.0


In [16]:
import pickle
fuzz_match = pickle.load(open( "menu_dict_fuzzy_match.p", "rb" ))




EOFError: Ran out of input